In [8]:
import os
import sys
import ast
from pprint import pprint
import pandas as pd
import matplotlib.pylab as plt
import matplotlib
import numpy as np
import datetime as dt
%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度

In [9]:
root = '/home/wmnlab/D/database'
# root = '/Users/jackbedford/Desktop/MOXA/Code/data'
paths = [s for s in os.listdir(root) if s.startswith('202')]

md_files = []

for date_dir in paths:
    datedir = os.path.join(root, date_dir)
    md_file = [os.path.join(datedir, s) for s in os.listdir(datedir) if s.endswith('.md')]
    md_files += md_file

md_files = sorted(md_files)

# Select dates
# md_files = md_files[:-1]
# md_files = [md_files[2]]

pprint(md_files)

['/home/wmnlab/D/database/2023-03-16/2023-03-16.md',
 '/home/wmnlab/D/database/2023-03-26/2023-03-26.md',
 '/home/wmnlab/D/database/2023-04-01/2023-04-01.md',
 '/home/wmnlab/D/database/2023-04-10/2023-04-10.md',
 '/home/wmnlab/D/database/2023-04-17/2023-04-17.md',
 '/home/wmnlab/D/database/2023-05-04/2023-05-04.md',
 '/home/wmnlab/D/database/2023-05-07/2023-05-07.md',
 '/home/wmnlab/D/database/2023-05-15/2023-05-15.md',
 '/home/wmnlab/D/database/2023-05-24/2023-05-24.md',
 '/home/wmnlab/D/database/2023-05-26/2023-05-26.md']


In [10]:
class Experiment():
    
    def __init__(self, dir_name, setting):
        
        self.dir_name = dir_name
        self.setting = setting
        
    def __repr__(self):
        return f'{self.dir_name} -> {self.setting}'

exp_list = []

for md_file in md_files:
    
    md_f = open(md_file, 'r')
    lines = []
    l = md_f.readline()[:-1]
    
    while l:
        
        if l == '#endif':
            break
        lines.append(l)
        l = md_f.readline()[:-1]
        
    md_f.close()
    
    for exp, dic in zip(lines[::2], lines[1::2]):
        
        dictionary = ast.literal_eval(dic)
        parent_dir = os.path.dirname(md_file)
        experiment = Experiment(dir_name = os.path.join(parent_dir, exp), setting=dictionary)
        exp_list.append(experiment)
        
pprint(exp_list)

[/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q -> {'qc00': 'B1', 'qc02': 'B7', 'qc03': 'B8'},
 /home/wmnlab/D/database/2023-03-26/Bandlock_Udp_All_RM500Q -> {'qc00': 'B3', 'qc02': 'B7', 'qc03': 'B8'},
 /home/wmnlab/D/database/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q -> {'qc00': 'B1B3', 'qc01': 'B1B8', 'qc02': 'LTE', 'qc03': 'All'},
 /home/wmnlab/D/database/2023-04-01/Bandlock_Udp_B1_B3_B7_B8_RM500Q -> {'qc00': 'B1', 'qc01': 'B3', 'qc02': 'B7', 'qc03': 'B8'},
 /home/wmnlab/D/database/2023-04-10/Experiment1 -> {'qc00': 'All', 'qc01': 'All', 'qc02': 'All', 'qc03': 'All'},
 /home/wmnlab/D/database/2023-04-10/Experiment2 -> {'qc00': 'B1', 'qc01': 'B3', 'qc02': 'B7', 'qc03': 'B8'},
 /home/wmnlab/D/database/2023-04-10/Experiment3 -> {'qc00': 'LTE', 'qc01': 'All', 'qc02': 'B7B8', 'qc03': 'B7'},
 /home/wmnlab/D/database/2023-04-17/Bandlock_Udp_All_LTE_All_LTE_RM500Q -> {'qc00': 'All', 'qc01': 'LTE', 'qc02': 'All', 'qc03': 'LTE'},
 /home/wmnlab/D/database/2023-04-17/

In [11]:
for exp in exp_list:
    dir_name = exp.dir_name
    devices = sorted(exp.setting.keys())
    for dev in devices:
        dev_folder = os.path.join(dir_name, dev)
        for trace in sorted(os.listdir(dev_folder)):
            if '#' not in trace:
                continue
            trace_folder = os.path.join(dev_folder, trace, 'data')
            for filename in os.listdir(trace_folder):
                if not filename.endswith('.csv'):
                    continue
                filepath = os.path.join(trace_folder, filename)
                print(filepath, os.path.isfile(filepath))
                print(f'{filepath[:-4]}.pkl')
                
                fp = pd.read_csv(filepath)
                fp.to_pickle(f'{filepath[:-4]}.pkl')

/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/udp_uplk_loss_latency.csv True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/udp_uplk_loss_latency.pkl
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/diag_log_qc00_2023-03-16_19-23-29_ml1.csv True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/diag_log_qc00_2023-03-16_19-23-29_ml1.pkl
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/diag_log_qc00_2023-03-16_19-23-29_nr_ml1.csv True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/diag_log_qc00_2023-03-16_19-23-29_nr_ml1.pkl
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/udp_dnlk_loss_latency.csv True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/udp_dnlk_loss_latency.pkl
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_

In [12]:
for exp in exp_list:
    dir_name = exp.dir_name
    devices = sorted(exp.setting.keys())
    for dev in devices:
        dev_folder = os.path.join(dir_name, dev)
        for trace in sorted(os.listdir(dev_folder)):
            if '#' not in trace:
                continue
            trace_folder = os.path.join(dev_folder, trace, 'data')
            for filename in os.listdir(trace_folder):
                if not filename.endswith('.pkl'):
                    continue
                filepath = os.path.join(trace_folder, filename)
                print(filepath, os.path.isfile(filepath))
                # print(f'{filepath[:-4]}.pkl')
                
                fp = pd.read_pickle(filepath)
                # fp.to_pickle(f'{filepath[:-4]}.pkl')

/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/diag_log_qc00_2023-03-16_19-23-29_ml1.pkl True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/udp_uplk_loss_latency.pkl True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/diag_log_qc00_2023-03-16_19-23-29_nr_ml1.pkl True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/udp_uplk_loss_latency_ho.pkl True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/udp_dnlk_loss_latency_ho.pkl True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/diag_log_qc00_2023-03-16_19-23-29_rrc.pkl True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/udp_uplk_integrate_sec.pkl True
/home/wmnlab/D/database/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/udp_dnlk_integrate_sec.pkl True
/home/wmnlab/D/database/2023-03-16/Bandlock_U